In [4]:
#import pathlib
from pathlib import Path
from typing import List, Tuple, Union
import csv
#import copy
import numpy as np
#import matplotlib.pyplot as plt
import serpentTools
from serpentTools.parsers.results import ResultsReader
#import click
#import matplotlib.pyplot as plt

serpentTools.settings.rc["serpentVersion"] = "2.1.32"
P = 48 #max no of FA
Z = 6  #max no of slices

AV = 0.6221408E+24

# get atomic wt numbers, store in atomic_wt_dict
with open("isotope_awt_list.csv", mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        atomic_wt_dict = {rows[0]:rows[1] for rows in reader}


def discharge_bu() -> None:
    res_list = []

    with serpentTools.settings.rc:
        serpentTools.settings.rc["verbosity"] = "error"
    dep = serpentTools.read("wh_lfr_dep.m")
    
    return dep

dep = discharge_bu()


def split_name(name: str)->Tuple[int, int]:
    """The functions take a fuel name, separates the P and Z and takes out the indexes
        Note: It s made only for this specific nomeclature, needs modification if you 
        want more fancy names

    Args:
        name (str): Takes in a standard fuel position name "fuelPxxxZxxx"

    Returns:
        Tuple[int, int]: Gives out a tuple with the val of P and Z
    """
    without_fuel = name[4:]
    split_without_fuel  = without_fuel.split("Z")
    p_num = int(split_without_fuel[0][1:])
    z_num = int(split_without_fuel[1])
    return p_num, z_num

### NEW VERSION

#### This is the part to be used inside the run to get the right _dep.m file

set inventory			% - Sets inventory for dep.m file 

Th-227 Th-228 Th-229 Th-230 Th-231 Th-232 Th-233 Th-234
Pa-231 Pa-232 Pa-233
U-232  U-233  U-234  U-235  U-236  U-237  U-238  U-239  U-240  U-241 
Np-235 Np-236 Np-237 Np-238 Np-239
Pu-236 Pu-237 Pu-238 Pu-239 Pu-240 Pu-241 Pu-242 Pu-243 Pu-244
Am-241 Am-242 Am-242m Am-243 Am-244 Am-244m 
Cm-240 Cm-241 Cm-242 Cm-243 Cm-244 Cm-245 Cm-246 Cm-247 Cm-248 Cm-249 Cm-250

Th Pa U Np Pu Am Cm			% - total list

In [5]:
materials = dep.materials.keys()
#create my own dict with fuel vol's
fuel_vol_dict = {}

# Go through each material index
for fuel_vol in materials:
    if fuel_vol == "total":
        # The continue, skips the loop interation at that point
        continue
    # Add the materials until 52 pos, exclude the total ones, big total and lost (maybe there is some way more intuitive)
    # Last column (last BU step in sss2) is taken and summed up for each fuel volume, store in dic wth key mdens_sum
    fuel_vol_dict[fuel_vol] = {"mdens_sum": dep.materials[fuel_vol].mdens[:, -1][0:52].sum(), "serpent_burnup": dep.materials[fuel_vol].burnup[-1]}

for fuel_vol in fuel_vol_dict:
    p_index, z_index = split_name(str(fuel_vol))
    mdens_to_divide_key = f"fuelP1Z{z_index}"
    mdens_to_divide = dep.materials[mdens_to_divide_key].mdens[:, -1][0:52].sum()
    ratio = fuel_vol_dict[fuel_vol]["mdens_sum"] / mdens_to_divide
    fuel_vol_dict[fuel_vol]["mdens_ratio"] = ratio
    #print(ratio)

for fuel_vol in fuel_vol_dict.keys():
    p_index, z_index = split_name(fuel_vol)
    burn_up = fuel_vol_dict[fuel_vol]["serpent_burnup"]
    #print(burn_up)
    if p_index == 1:
        fuel_vol_dict[fuel_vol]["corrected_burnup"] = burn_up
        continue
    
    material_name = f"fuelP{p_index - 1}Z{z_index}"
    ratio = fuel_vol_dict[material_name]["mdens_ratio"]
    corrected_burnup = burn_up * ratio
    fuel_vol_dict[fuel_vol]["corrected_burnup"] = corrected_burnup

average_burnups = {}
for fuel_vol in fuel_vol_dict.keys():
    p_index, _ = split_name(fuel_vol)
    average_burnup = 0
    for zidx in range(1 , Z + 1):
        key_construct = f"fuelP{p_index}Z{zidx}"
        average_burnup += fuel_vol_dict[key_construct]["corrected_burnup"] / Z
    average_burnups[f"P{p_index}"] = average_burnup

average_burnups_axial = {}
for fuel_vol in fuel_vol_dict.keys():
    _, z_index = split_name(fuel_vol)
    average_axial_burnup = 0
    for pidx in range(1 , P + 1):
        key_construct = f"fuelP{pidx}Z{z_index}"
        average_axial_burnup += fuel_vol_dict[key_construct]["corrected_burnup"]
    average_burnups_axial[f"Z{z_index}"] = average_axial_burnup


In [6]:
average_burnups_axial

{'Z1': 216.21975823721365,
 'Z2': 310.3409709876174,
 'Z3': 361.6385650102795,
 'Z4': 358.2488223888461,
 'Z5': 306.70536793918916,
 'Z6': 214.36409303289037}

In [7]:
sum([value for key, value in average_burnups.items()])


294.58626293267275

In [8]:
sum([value for key, value in average_burnups_axial.items()]) / Z

294.5862629326727

In [9]:
average_burnups

{'P48': 3.4215320397990805,
 'P47': 6.013152438488769,
 'P46': 4.660235440363667,
 'P45': 3.5966143123350327,
 'P44': 5.564641105938985,
 'P43': 7.627093295813536,
 'P42': 9.456449846435365,
 'P41': 11.333798834398877,
 'P40': 8.723655444245587,
 'P39': 6.025203380540794,
 'P38': 12.158465609152554,
 'P37': 15.144436179254802,
 'P36': 13.701099670537346,
 'P35': 20.500858134385567,
 'P34': 22.703654313646545,
 'P33': 21.3809226379891,
 'P32': 18.54022827734024,
 'P31': 18.20333181509227,
 'P30': 15.086422453318026,
 'P29': 14.043377858226968,
 'P28': 10.054335086481636,
 'P27': 10.030427543114394,
 'P26': 6.678861360607794,
 'P25': 5.686642638912169,
 'P24': 3.2772566475938643,
 'P23': 3.48477804871016,
 'P22': 3.6264711466647883,
 'P21': 2.176349211582113,
 'P20': 1.844711928635799,
 'P19': 1.299736142192507,
 'P18': 0.6109566753171612,
 'P17': 0.7508937213422364,
 'P16': 0.8108628429441427,
 'P15': 0.8597394122446307,
 'P14': 0.8579499338198338,
 'P13': 0.7876727418533164,
 'P12': 0.

In [11]:
fuel_vol_dict

{'fuelP48Z1': {'mdens_sum': 8.729094239329758,
  'serpent_burnup': 3.69163,
  'mdens_ratio': 0.7782556112744137,
  'corrected_burnup': 2.8902787392792604},
 'fuelP48Z2': {'mdens_sum': 7.659742187033028,
  'serpent_burnup': 5.12255,
  'mdens_ratio': 0.6829325193693492,
  'corrected_burnup': 3.5278937598795124},
 'fuelP48Z3': {'mdens_sum': 7.095437810742455,
  'serpent_burnup': 6.08486,
  'mdens_ratio': 0.632632566554351,
  'corrected_burnup': 3.861003896327579},
 'fuelP48Z4': {'mdens_sum': 7.100892631613838,
  'serpent_burnup': 5.76139,
  'mdens_ratio': 0.6331214558332143,
  'corrected_burnup': 3.6622019679237856},
 'fuelP48Z5': {'mdens_sum': 7.682356193954687,
  'serpent_burnup': 5.29506,
  'mdens_ratio': 0.6849524099576728,
  'corrected_burnup': 3.650562608833183},
 'fuelP48Z6': {'mdens_sum': 8.7639151237853,
  'serpent_burnup': 3.75304,
  'mdens_ratio': 0.781360683288206,
  'corrected_burnup': 2.9372512665511623},
 'fuelP47Z1': {'mdens_sum': 8.781495500540725,
  'serpent_burnup': 6.3